In [1]:
!pip install pydicom

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import tensorflow as tf
import  tensorflow_hub as hub
import numpy as np
import glob
import pydicom as py
import cv2
import matplotlib.pyplot as plt

from PIL import Image
from google.colab.patches import cv2_imshow

In [4]:
root_dir = 'drive/My Drive/Lab'

In [5]:
output_dir = os.path.join(root_dir, 'output/')

model_file = os.path.join(output_dir, 'model.h5')

In [6]:
model = tf.keras.models.load_model(
    model_file,
    custom_objects={
        'KerasLayer': hub.KerasLayer
    }
)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1280)              1639680   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 540)               691740    
_________________________________________________________________
dropout_2 (Dropout)          (None, 540)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1

---
---

In [8]:
def predict(np_image, model, top_k=1):
    
    image_tensor = tf.image.convert_image_dtype(np_image, dtype=tf.float32)

    np_img = tf.image.resize(image_tensor, [224, 224]).numpy()

    image = tf.expand_dims(np_img, 0)
    
    result = model.predict(image)

    predicted_id = np.argmax(result, axis=-1)

    return predicted_id[0]

In [9]:
def predict_dcm_path(dcm_path, model, top_k=1):
    
    np_image = py.dcmread(dcm_path).pixel_array
    
    return predict(np_image, model, top_k)

In [10]:
def predict_jpg_path(jpg_path, model, top_k=1):

    np_image = cv2.imread(jpg_path, 0)

    np_image = cv2.cvtColor(np_image, cv2.COLOR_GRAY2RGB)
    
    return predict(np_image, model, top_k)

---
---

In [11]:
image_file_paths = glob.glob('/{}/PNEUMONIA(*).jpg'.format('content'))

In [12]:
correct_positive = 0

for filepath in image_file_paths:
  if (predict_jpg_path(filepath, model) == 1):
    correct_positive += 1

print(correct_positive / len(image_file_paths))

0.8103448275862069


In [13]:
image_file_paths = glob.glob('/{}/NORMAL(*).jpg'.format('content'))

In [14]:
correct_negative = 0

for filepath in image_file_paths:
  if (predict_jpg_path(filepath, model) == 0):
    correct_negative += 1

print(correct_negative / len(image_file_paths))

0.8
